In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import numpy_financial as npf

In [2]:
current_loan_amount = 300000
current_loan_percent = 3.5
current_loan_term = 15
current_loan_start = '06/01/2017'
payments_per_year = 12

new_loan_percent = 2.5
new_loan_term = 15
new_loan_start = '03/01/2021' 
new_loan_fees = 5000

assumed_rate_of_return = 1.07

In [3]:
rng = pd.date_range(start = pd.to_datetime(current_loan_start, format = '%m/%d/%Y'), periods=current_loan_term*payments_per_year, freq='MS')
df = pd.DataFrame({ 'Date': rng, 
                    'Beginning_Balance':0,
                    'Scheduled_Payment':0,
                    'Principal':0,
                    'Interest':0,
                    'Ending_Balance':0,
                    'Cumulative_Interest':0                    
                  }) 

In [4]:
df["Scheduled_Payment"] = -npf.pmt(current_loan_percent/100/payments_per_year, current_loan_term*payments_per_year, current_loan_amount)
df["Interest"] = -npf.ipmt(current_loan_percent/100/payments_per_year, df.index+1, current_loan_term*payments_per_year, current_loan_amount)
df["Principal"] = df["Scheduled_Payment"]-df["Interest"]
df['Cumulative_Interest'] = df['Interest'].cumsum()
df['Ending_Balance'] = current_loan_amount-df['Principal'].cumsum()
df['Beginning_Balance'] = df['Principal'] + df['Ending_Balance']

In [5]:
pd.options.display.float_format = '${:,.2f}'.format

In [6]:
new_loan_amount = df[df['Date'] == pd.to_datetime(new_loan_start, format = '%m/%d/%Y')]['Beginning_Balance'].iloc[0]

nrng = pd.date_range(start = pd.to_datetime(new_loan_start, format = '%m/%d/%Y'), periods=new_loan_term*payments_per_year, freq='MS')
ndf = pd.DataFrame({ 'Date': nrng, 
                    'Beginning_Balance':0,
                    'Scheduled_Payment':0,
                    'Principal':0,
                    'Interest':0,
                    'Ending_Balance':0,
                    'Cumulative_Interest':0                    
                  })

In [7]:
ndf["Scheduled_Payment"] = -npf.pmt(new_loan_percent/100/payments_per_year, new_loan_term*payments_per_year, new_loan_amount)
ndf["Interest"] = -npf.ipmt(new_loan_percent/100/payments_per_year, ndf.index+1, new_loan_term*payments_per_year, new_loan_amount)
ndf["Principal"] = ndf["Scheduled_Payment"]-ndf["Interest"]
ndf['Cumulative_Interest'] = ndf['Interest'].cumsum()
ndf['Ending_Balance'] = new_loan_amount-ndf['Principal'].cumsum()
ndf['Beginning_Balance'] = ndf['Principal'] + ndf['Ending_Balance']

In [8]:
pd.set_option('display.max_rows', None)
finaldf = df.merge(ndf, how = 'outer', on='Date', suffixes = ('_cur', '_new') )
finaldf = finaldf[finaldf['Date'] >= pd.to_datetime(new_loan_start, format = '%m/%d/%Y')]
finaldf = finaldf.reset_index(drop = True)
finaldf = finaldf.fillna(0)
finaldf['Differnce_Paid'] = finaldf['Scheduled_Payment_new'] - finaldf['Scheduled_Payment_cur']
finaldf['Differnce_Paid_cum'] = finaldf['Differnce_Paid'].cumsum()
finaldf['Owed_Balance_Diff'] = finaldf['Ending_Balance_new'] - finaldf['Ending_Balance_cur']
finaldf['Refinance_fee'] = 0
finaldf.loc[0, 'Refinance_fee'] = new_loan_fees
finaldf['money_in_hand'] = -finaldf['Refinance_fee'].cumsum() - finaldf['Differnce_Paid_cum']
finaldf['HouseVal_and_netpaiddiff_cum'] = -finaldf['Owed_Balance_Diff'] + finaldf['money_in_hand']
finaldf['Interest_Potential'] = finaldf['money_in_hand']*assumed_rate_of_return**(1/12)-finaldf['money_in_hand']
finaldf['Interest_Potential_cum'] = finaldf['Interest_Potential'].cumsum()

In [9]:
finaldf.head()

,Date,Beginning_Balance_cur,Scheduled_Payment_cur,Principal_cur,Interest_cur,Ending_Balance_cur,Cumulative_Interest_cur,Beginning_Balance_new,Scheduled_Payment_new,Principal_new,...,Ending_Balance_new,Cumulative_Interest_new,Differnce_Paid,Differnce_Paid_cum,Owed_Balance_Diff,Refinance_fee,money_in_hand,HouseVal_and_netpaiddiff_cum,Interest_Potential,Interest_Potential_cum
0,2021-03-01,"$239,041.68","$2,144.65","$1,447.44",$697.20,"$237,594.23","$36,248.03","$239,041.68","$1,593.90","$1,095.90",...,"$237,945.78",$498.00,$-550.74,$-550.74,$351.54,5000,"$-4,449.26","$-4,800.80",$-25.16,$-25.16
1,2021-04-01,"$237,594.23","$2,144.65","$1,451.66",$692.98,"$236,142.57","$36,941.01","$237,945.78","$1,593.90","$1,098.18",...,"$236,847.59",$993.72,$-550.74,"$-1,101.49",$705.02,0,"$-3,898.51","$-4,603.54",$-22.04,$-47.20
2,2021-05-01,"$236,142.57","$2,144.65","$1,455.90",$688.75,"$234,686.67","$37,629.76","$236,847.59","$1,593.90","$1,100.47",...,"$235,747.12","$1,487.16",$-550.74,"$-1,652.23","$1,060.45",0,"$-3,347.77","$-4,408.22",$-18.93,$-66.13
3,2021-06-01,"$234,686.67","$2,144.65","$1,460.14",$684.50,"$233,226.53","$38,314.26","$235,747.12","$1,593.90","$1,102.76",...,"$234,644.36","$1,978.30",$-550.74,"$-2,202.97","$1,417.83",0,"$-2,797.03","$-4,214.86",$-15.81,$-81.94
4,2021-07-01,"$233,226.53","$2,144.65","$1,464.40",$680.24,"$231,762.12","$38,994.50","$234,644.36","$1,593.90","$1,105.06",...,"$233,539.30","$2,467.14",$-550.74,"$-2,753.72","$1,777.17",0,"$-2,246.28","$-4,023.45",$-12.70,$-94.64


In [10]:
fig = px.line(
            finaldf, 
            x="Date", 
            y=['Differnce_Paid_cum', 'money_in_hand', 'HouseVal_and_netpaiddiff_cum', 'Interest_Potential_cum'],
            title= 'Test',
            template='none'
            )
fig.show()

In [11]:
finaldf.columns

Index(['Date', 'Beginning_Balance_cur', 'Scheduled_Payment_cur',
       'Principal_cur', 'Interest_cur', 'Ending_Balance_cur',
       'Cumulative_Interest_cur', 'Beginning_Balance_new',
       'Scheduled_Payment_new', 'Principal_new', 'Interest_new',
       'Ending_Balance_new', 'Cumulative_Interest_new', 'Differnce_Paid',
       'Differnce_Paid_cum', 'Owed_Balance_Diff', 'Refinance_fee',
       'money_in_hand', 'HouseVal_and_netpaiddiff_cum', 'Interest_Potential',
       'Interest_Potential_cum'],
      dtype='object')